In [1]:
### Load the auto reload extension to automatically reload modules when files on disk are updated
%load_ext autoreload
# it will automatically be reloaded without the need to restart the kernel.
%autoreload 2

In [2]:
import sys
sys.path.append("/agrilearn_app/agrilearn/submodules/commons/")

In [28]:
import boto3
import os
import geopandas as gpd
import pandas as pd
import folium
import numpy as np
from folium import GeoJson
from io import BytesIO
from tqdm import tqdm
from eolearn.core import EOPatch
from glob import glob
from agrilearn.crop_classification import evaluate_utils as eval_util
from agrilearn.commons.io.read_transform import read_multiplies_files_from_path, transform_pandas_to_geopandas

In [5]:
from agrilearn.crop_classification import yaml_utils, processing
from agrilearn.crop_classification.cropclassifier import CropClassifier

### Global Variables

In [11]:
YAML_CONFIG_PATH = "/agrilearn_app/output/config/experiment_12.yaml"
config = yaml_utils.load_config(YAML_CONFIG_PATH)

In [12]:
index = 0
DATASET_NAME = config['evaluate'][index]['dataset_name']
GEOPACKAGE_PATH = config['evaluate'][index]['geopackage_test_data_path']
EOPATCH_PATH = config['evaluate'][index]['eopatch_folder']
LABEL_MONITORING_CLASS = config['evaluate'][index]['label_true']
LABEL_EOPATCH_LOCATION = config['evaluate'][index]['label_eopatch_location']
EXPERIMENT_PATH = config["data"]["experiment_path"]

# Adicionando prints para depuração
print(f"Index: {index}")
print(f"DATASET_NAME: {DATASET_NAME}")
print(f"GEOPACKAGE_PATH: {GEOPACKAGE_PATH}")
print(f"EOPATCH_PATH: {EOPATCH_PATH}")
print(f"LABEL_MONITORING_CLASS: {LABEL_MONITORING_CLASS}")
print(f"LABEL_EOPATCH_LOCATION: {LABEL_EOPATCH_LOCATION}")


Index: 0
DATASET_NAME: banco_do_brasil
GEOPACKAGE_PATH: /agrilearn_app/datasets/baselines/banco_do_brasil/geopackage/SOYBEAN_5973.gpkg
EOPATCH_PATH: /agrilearn_app/datasets/baselines/banco_do_brasil/eopatch/processed/
LABEL_MONITORING_CLASS: monitoring_class
LABEL_EOPATCH_LOCATION: eopath_location


# 1. Read Datasets

In [13]:
gdf = gpd.read_file(GEOPACKAGE_PATH)

In [14]:
eval_util.get_dataset_distribution(gdf)

,count,proportion
monitoring_class,,
SOYBEAN,5973,1.0


In [15]:
print(f"There are {gdf.shape[0]} rows and {gdf.shape[1]} columns")

There are 5973 rows and 36 columns


# 2. Predict DEBUG

In [19]:
OUTPUT_PREDICTIONS = os.path.join(EXPERIMENT_PATH, "predictions")
OUTPUT_RESULTS = os.path.join(EXPERIMENT_PATH, "results")
OUTPUT_PREDICTIONS_DATASET = os.path.join(OUTPUT_PREDICTIONS, DATASET_NAME)
OUTPUT_RESULTS_DATASET = os.path.join(OUTPUT_RESULTS, DATASET_NAME)
print(f"Creating and checking prediction path: {OUTPUT_PREDICTIONS_DATASET}")
os.makedirs(OUTPUT_PREDICTIONS_DATASET, exist_ok=True)
print(f"Creating and checking results path: {OUTPUT_RESULTS_DATASET}")
os.makedirs(OUTPUT_RESULTS_DATASET, exist_ok=True)

Creating and checking prediction path: /agrilearn_app/output/experiment_12/predictions/banco_do_brasil
Creating and checking results path: /agrilearn_app/output/experiment_12/results/banco_do_brasil


In [21]:
config = yaml_utils.load_config(YAML_CONFIG_PATH)

crop_classifer = CropClassifier(YAML_CONFIG_PATH)

In [24]:
df_prediction = read_multiplies_files_from_path(OUTPUT_PREDICTIONS_DATASET, 
                                                sep=",")

There are 5952 files to read in path: /agrilearn_app/output/experiment_12/predictions/banco_do_brasil


Reading files:: 100%|██████████| 5952/5952 [00:31<00:00, 189.73it/s]


In [32]:
df_prediction['obs_crop_rnn'].iloc[0]

"NoneError in crop classification: name 'parser' is not defined; "

In [26]:
# for i in tqdm(range(len(gdf.sample(2))), desc='Processing:'):
#     try:
#         row = gdf.loc[[i]]  # Select the row as a GeoDataFrame
#         location_id = row.loc[i, LABEL_EOPATCH_LOCATION]

#         final_path = os.path.join(OUTPUT_PREDICTIONS, LABEL_EOPATCH_LOCATION)

#         if os.path.exists(final_path):
#             if debug:
#                 print(f"skipped prediction path: {final_path}")
#             continue
#         else:
#             eopath_id_path = os.path.join(
#                 EOPATCH_PATH,
#                 location_id,
#                 "eopatch_0_col-0_row-0")
#             eopatch = EOPatch.load(eopath_id_path, lazy_loading=True)

#             if eopatch is None:
#                 raise ValueError(f"EOPatch is null for path: {eopath_id_path}")

#             result = crop_classifer.predict(eop=eopatch,
#                                             input_gdf=row)

#             result.to_csv(final_path, index=False)
#     except Exception as e:
#         print(f"Failed: {e}")

In [21]:
location_id

NameError: name 'location_id' is not defined

In [ ]:
row

In [ ]:
"start_2022-08-01_end_2023-07-01_monitoring_class_CORN_epsg4326_minxymaxxy_-52dot754283709825316_-29dot76146485746487_-52dot750590619481365_-29dot757761764799973"

In [ ]:
row['eopath_location'].values

In [ ]:
bands_npy = eopatch.data["BANDS"]

In [ ]:
cloud_mask = eopatch.mask["S2_CLOUDLESS"].squeeze().astype(bool)

In [ ]:
aoi_mask = eopatch.mask_timeless["AOI_MASK"].squeeze().astype(bool)

In [ ]:
eopatch.mask['S2_CLOUDLESS']

In [ ]:
eopatch.data["NDVI"].shape

In [ ]:
import rasterio
from rasterio.features import geometry_mask

In [ ]:
eop_shape = eopatch.data["NDVI"].shape

transform = rasterio.transform.from_bounds(*eopatch.bbox, 
                               eop_shape[2], 
                               eop_shape[1])

In [ ]:
geo = row.geometry.iloc[0]

In [ ]:
geo

In [ ]:
mask = geometry_mask(
        [geo], eop_shape[1:3], transform, all_touched=True, invert=True
    )

In [ ]:
mask = mask[:, :, None].astype(bool)

In [ ]:
eopatch.mask_timeless["AOI_MASK"] = mask

In [ ]:
mask = eopatch.mask_timeless["AOI_MASK"]
mask

In [ ]:
ask = np.tile(mask, (len(eopatch.mask["S2_CLOUDLESS"]), 1, 1, 1))

In [ ]:
eopatch.mask["S2_CLOUDLESS"] = eopatch.mask["S2_CLOUDLESS"] * mask
eopatch.mask["S2_CLOUDLESS"] = eopatch.mask["S2_CLOUDLESS"].astype(bool)

In [ ]:
from agrilearn.commons.series_utils.smoothing_filters import NdviTimeSeriesSavitzkyGolayFilter

In [ ]:
golayfilter = NdviTimeSeriesSavitzkyGolayFilter()

In [ ]:
series_s2cloud_with_mask_task = NDVIEOpatchToPandasSeries("S2_CLOUDLESS")

In [ ]:
result = crop_classifer.predict(eop=eopatch,
                                input_gdf=row)

In [ ]:
bands_npy = eopatch.data["BANDS"]

In [ ]:
bands_npy